In [8]:
import pandas as pd 
df=pd.read_csv("../artifacts/data_ingestion/data.csv")
df.head()

,car_name,registration_year,insurance_validity,fuel_type,ownsership,transmission,manufacturing_year,seats,kms_driven,mileage(kmpl),engine(cc),torque(Nm),price(in lakhs),short_carname
0,2017 Mercedes-Benz S-Class S400,2017,Comprehensive,Petrol,First Owner,Automatic,2017,5.0,56000.0,7.81,2996.0,333.0,63.75,Mercedes-Benz S-Class
1,2020 Nissan Magnite Turbo CVT XV Premium Opt BSVI,2021,Comprehensive,Petrol,First Owner,Automatic,2020,5.0,30615.0,17.40,999.0,9863.0,8.99,Nissan Magnite
2,2018 BMW X1 sDrive 20d xLine,2018,Comprehensive,Diesel,First Owner,Automatic,2018,5.0,24000.0,20.68,1995.0,188.0,23.75,BMW X1
3,2019 Kia Seltos GTX Plus,2019,Comprehensive,Petrol,First Owner,Manual,2019,5.0,18378.0,16.50,1353.0,13808.0,13.56,Kia Seltos
4,2019 Skoda Superb LK 1.8 TSI AT,2019,Comprehensive,Petrol,First Owner,Automatic,2019,5.0,44900.0,14.67,1798.0,17746.0,24.00,Skoda Superb


In [10]:
df.dtypes

car_name               object
registration_year       int64
insurance_validity     object
fuel_type              object
ownsership             object
transmission           object
manufacturing_year      int64
seats                 float64
kms_driven            float64
mileage(kmpl)         float64
engine(cc)            float64
torque(Nm)            float64
price(in lakhs)       float64
short_carname          object
dtype: object

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [23]:
from src.carPricePrediction.constants import *
from src.carPricePrediction.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [24]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            data_columns = list(data.columns)
            data_dtypes =  data.dtypes.astype(str).tolist()

            schema = self.config.all_schema
            schema_cols = self.config.all_schema.keys()

            for col, dtype in zip(data_columns, data_dtypes):
                if (col not in schema_cols) or (schema.get(col) != dtype):
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        break
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [25]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-06-28 16:54:19,637: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-28 16:54:19,783: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 16:54:19,787: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-28 16:54:19,789: INFO: common: created directory at: artifacts]
[2024-06-28 16:54:19,791: INFO: common: created directory at: artifacts/data_validation]


In [22]:
import os
os.chdir("../")